# RandomForest를 활용한 자전거 대여량 예측하기

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import datetime

In [ ]:
from tensorflow.keras import datasets, layers, models

In [ ]:
path = "data/"
print(os.listdir("data/"))

# load data

In [ ]:
df_train_eda = pd.read_csv(path + "train.csv", parse_dates = ['datetime'],
                         index_col='datetime', infer_datetime_format=True)
df_test = pd.read_csv(path + 'test.csv', parse_dates = ['datetime'],
                        index_col='datetime', infer_datetime_format=True)
df_submission = pd.read_csv(path+'sampleSubmission.csv', parse_dates = ['datetime'],
                        index_col='datetime', infer_datetime_format=True)

# Columns 

- datetime : 시간별 날짜
- season :  
  1.(1분기)  
  2.(2분기)   
  3.(3분기)  
  4.(4분기)
- holiday : 하루가 휴일로 간주되는지 여부
- workingday : 주말과 휴일이 아닌 일하는 날
- weather :   
  1.(맑음, 구름, 조금, 흐림)  
  2.(안개+흐림, 안개+구름, 안개+구름이 거의 없음 + 흐림)  
  3.(가벼운 눈, 가벼운 비 + 천둥 + 구름, 가벼운 비 + 구름)  
  4.(폭우 + 우박 + 천둥 + 안개, 눈 + 안개)
- temp : 섭씨 온도
- atemp : 섭씨 온도의 느낌
- humidity : 상대 습도
- windspeed : 풍속
- casual : 미등록 사용자 대여수
- registered : 등록된 사용자 대여수
- count : 대여수

 

In [ ]:
df_train_eda.head(3)

In [ ]:
df_test.head(3)

In [ ]:
df_submission.head(3)

# EDA

## 데이터 타입 확인

In [ ]:
print(df_train_eda.shape, df_test.shape)
print("훈련 데이터")
print(df_train_eda.dtypes)
print("테스트 데이터")
print(df_test.dtypes)

## 결측치 확인

In [ ]:
print(df_train_eda.isnull().sum())

In [ ]:
print(df_test.isnull().sum())

In [ ]:
print(df_submission.isnull().sum())

## datetime 에서 년,월,일,시간,분,초를 추출해 column 추가

In [ ]:
df_train_eda['year'] = df_train_eda.index.year

In [ ]:
df_train_eda['year'] = df_train_eda.index.year
df_train_eda['month'] = df_train_eda.index.month
df_train_eda['day'] = df_train_eda.index.day
df_train_eda['hour'] = df_train_eda.index.hour
df_train_eda['minute'] = df_train_eda.index.minute
df_train_eda['second'] = df_train_eda.index.second

In [ ]:
df_train_eda.head()

## 연도별, 월별,시간별에 따른 대여량 평균치 분석

In [ ]:
def bar_plot(df, x, ax):
    fig = plt.figure(figsize=(5,3))
    sns.barplot(data=df, x=x, y="count", palette="Blues_d", ax=ax)

In [ ]:
figure, ((ax1, ax2, ax3), (ax4, ax5, ax6)) = plt.subplots(nrows=2, ncols=3)
figure.set_size_inches(18, 10)

bar_plot(df_train_eda, "year", ax=ax1)
bar_plot(df_train_eda, "month", ax=ax2)
bar_plot(df_train_eda, "day", ax=ax3)
bar_plot(df_train_eda, "hour", ax=ax4)
bar_plot(df_train_eda, "minute", ax=ax5)
bar_plot(df_train_eda, "second", ax=ax6)

## 연도별,월별,시간별에 따른 분석 결과

- 연도별 : 2011년보다 2012년 대여량이 많아짐  
- 월별 : 월별 대여량은 6월에 가장 많고, 따뜻한 계절(5~10월달)에 대여량이 많음
- 일별 : 일별 대여량은 크게 차이점이 없고, 특징점 역시 없음.
- 시간별 : 오전에는 8시에 가장 많고, 오후에는 17시~18시에 가장 많음

## datetime을 기반으로 요일 추출

In [ ]:
df_train_eda['dayofweek'] = df_train_eda.index.dayofweek
df_train_eda.head(3)

## 시간대별 자전거 대여량 (근무일 유무, 요일, 시즌, 날씨)

In [ ]:
def point_plot(df, hue, ax):
    sns.pointplot(data=df, x="hour", y="count", ax=ax, hue=hue)

In [ ]:
fig,(ax1, ax2, ax3, ax4)= plt.subplots(nrows=4)
fig.set_size_inches(18,25)

#sns.pointplot(df_train, ax=ax1)
point_plot(df_train_eda, 'workingday', ax=ax1)
point_plot(df_train_eda, 'dayofweek', ax=ax2)
point_plot(df_train_eda, 'season', ax=ax3)
point_plot(df_train_eda, 'weather', ax=ax4)

앞선 시간대별 자전거 대여량 그래프를 보면 오전8시, 오후 5~6시에 가장 대여량이 많았다.

근무일, 요일, 분기, 날씨에 따른 대여량에 대한 분석  
- 근무일에는 출근시간(8시), 퇴근시간(17 ~ 18시)에 가장 대여량이 높았고, 휴무일에는 12 ~ 16시에 가장 대여를 많이했다.
- 평일은 근무일 대여량, 주말은 휴무일 대여량에 따르는 것을 확인할 수 있다.
- 3분기(7 ~ 9월)에 가장 대여를 많이하고, 1분기(1 ~ 3월)에 가장 적게 대여를 하는 것을 볼 수 있다.
- 날씨가 좋을 수록 대여를 많이하고, 좋지 않을 때는 대여를 많이 안한다.

# EDA 분석 끝 다시 원본 데이터 불러오기

In [ ]:
df_train = pd.read_csv(path + "train.csv", parse_dates = ['datetime'],
                       index_col='datetime', infer_datetime_format=True)

In [ ]:
df_train.shape

In [ ]:
df_train.tail()

### df_train데이터는 한 시간 단위로 2011-01-01 00:00:00  ~ 2012-12-19 23 23:00:00 ( 1일부터19일)
### 즉, 2(years) * 12(month) * 19(days) * 24(hours) =  10944(timestep)이어야한다.
### 하지만 df_train데이터는 10886(timestep) 이므로 중간에 누락된 정보가 있다.
### 따라서 (10944-10886) = 58개의 timestep을 채워야 한다.

In [ ]:
num_months_per_year = 12
year_list = [2011, 2012]

In [ ]:
df_train_temp = pd.DataFrame(columns=df_train.columns)

for year in year_list: # 2011 2012
    for month in range(num_months_per_year):# 0~11
        start_date = datetime.datetime(year, month+1, 1, 0, 0, 0)
        end_date = datetime.datetime(year, month+1, 19, 23, 0, 0)
        temp = df_train[start_date:end_date].resample('H').asfreq()
        df_train_temp = df_train_temp.append(temp)
        
train_data = df_train_temp

In [ ]:
train_data.shape

### 채워준 timestep의 결측치를 채워야한다.

In [ ]:
train_data.isna().sum()

In [ ]:
null_feature = train_data[train_data['count'].isnull()].index
null_feature

### season, holiday, workingday, weather들은 backfill로 채운다.(같은 날짜 이기 때문)

In [ ]:
backfill_features = ['season', 'holiday', 'workingday', 'weather']
train_data[backfill_features] = train_data[backfill_features].fillna(method='backfill')

### temp, atemp, humidity, windspeed 들은 linear로 채운다.

In [ ]:
fill_linear_features = ['temp', 'atemp', 'humidity', 'windspeed']
train_data[fill_linear_features] = train_data[fill_linear_features].interpolate(method='linear')

### 문제는 casual, registered, count들이다.
### target값과 연관된 column이기 때문에 머신러닝 기법을 활용해 채운다.
### casual, registered 들의 합은 count이기 때문에 drop한다.

In [ ]:
null_df = train_data.loc[null_feature]
null_df = null_df.drop("casual",axis=1)
null_df = null_df.drop("registered",axis=1)
null_df.head() # 

In [ ]:
full_df = train_data.drop(index = null_feature)
full_df = full_df.drop("casual",axis=1)
full_df = full_df.drop("registered",axis=1)
full_df.head() 

## EDA분석에서 시간대별 대여량에 영향이 있는  column을 갖고온다.

In [ ]:
null_df['year'] = null_df.index.year
null_df['month'] = null_df.index.month
null_df['day'] = null_df.index.day
null_df['hour'] = null_df.index.hour

full_df['year'] = full_df.index.year
full_df['month'] = full_df.index.month
full_df['day'] = full_df.index.day
full_df['hour'] = full_df.index.hour

In [ ]:
X = full_df.drop("count", axis=1)
X

In [ ]:
y = full_df["count"]
y

## randomforest 학습

In [ ]:
from sklearn.model_selection import train_test_split

### 데이터 split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

### 필요한 라이브러리

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV  # 매개변수를 자동으로 변수 설정해서 최적 찾기 위한 방법
from sklearn.model_selection import KFold  # 몇번 나눌지
from sklearn.metrics import r2_score, mean_squared_error  # SVR 모델의 예측과 실제의 결과를 확인하는 성능지표

def rmsle(y,pred):
    log_y = np.log1p(y)
    log_pred = np.log1p(pred)
    squared_error = (log_y - log_pred)**2
    rmsle = np.sqrt(np.mean(squared_error))
    return rmsle

In [ ]:
seed = 2021

### random_grid 파라미터 값 설정

In [ ]:
"""
# 랜덤 포레스트의 트리 수 
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1000, num = 100)] 
# 모든 분할에서 고려해야 할 기능의 수 
max_features = ['auto', ' qrt '] 
# 트리의 최대 레벨 수 
max_depth = [int(x) for x in np.linspace(10, 11, num = 11)] 
max_depth.append(None) 
# 노드를 분할하는 데 필요한 최소 샘플 수 
min_samples_split = [2, 5, 10] 
# 각 리프 노드에 필요한 최소 샘플 수 
min_samples_leaf = [1, 2, 4] 
# 각 트리 
# 부트스트랩 훈련을 위한 샘플 선택 방법 = [True, False]
# 랜덤 그리드 생성 
random_grid = {'n_estimators': n_estimators, 
               'max_features': max_features, 
               'max_depth': max_depth, 
               'min_samples_split': min_samples_split, 
               'min_samples_leaf': min_samples_leaf, 
               }"""

In [ ]:
random_grid = {'n_estimators': [100], 
               'max_features': ["auto"], 
               'max_depth': [10], 
               'min_samples_split': [2], 
               'min_samples_leaf': [1], 
               }

In [ ]:
rf = RandomForestRegressor(random_state=seed, bootstrap=True)
cv = KFold(n_splits = 5, shuffle = True, random_state=seed)

grid=GridSearchCV(estimator=rf,
                 param_grid=random_grid,
                 cv=cv)

grid.fit(X_train, y_train)

## best param 값 

In [ ]:
best_parameters =grid.best_params_
print(best_parameters)

## final model에 다시 적용

In [ ]:
final_model=RandomForestRegressor(**best_parameters, random_state=seed)
final_model.fit(X_train, y_train)

predict = final_model.predict(X_test)

In [ ]:
Score=r2_score(y_test,predict)
Rmsle = rmsle(y_test,predict)
print("r2 score:{:0.3f}, rmsle:{:0.3f}".format(Score, Rmsle))

# 이후로는 RNN

In [ ]:
"""null_df_train = null_df.drop("count", axis=1)
null_df_test_predict = final_model.predict(null_df_train)
null_df["count"] = null_df_test_predict
null_df["count"] = round(null_df["count"])
data = pd.concat([null_df, full_df]).sort_index(ascending=True)
drop_features = ['year','month','day','hour']
data = data.drop(drop_features, axis=1)
data.to_csv("train_2.csv")"""

In [ ]:
# 이제 이 데이터로 RNN을 학습함 
# data.to_csv("pre_processing_train.csv")